# Cleaning of the database

In [106]:
import requests
import pandas as pd

data = pd.read_csv (r"SRC/2018.csv", encoding='latin-1')
df = pd.DataFrame(data)
display(df.head())
print(df.shape)




,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2018-01-01,UA,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,...,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,UA,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,...,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,UA,2426,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,...,134.0,126.0,106.0,846.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,UA,2425,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,...,190.0,182.0,157.0,1120.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-01,UA,2424,ORD,ALB,630,650.0,20.0,13.0,703.0,...,112.0,106.0,83.0,723.0,NaN,NaN,NaN,NaN,NaN,NaN


(7213446, 28)


In [184]:
#columns inspection
df.columns
 


Index(['FL_DATE', 'OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'Unnamed: 27'],
      dtype='object')

# Removing of the duplicates and of the null elements

In [185]:
# Removing the unusefull columns -'Unnamed 27' is empty

df_clean=df.drop(['Unnamed: 27','TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON', 'TAXI_IN','CANCELLATION_CODE'],axis=1)

#dropping unusefull rows    
df_clean=df_clean.dropna(axis=0, how='all', thresh=None, subset=None, inplace=False)

#removing of eventual duplicates
df_clean=df_clean.drop_duplicates()

#Replacing of the null values with 0
df_clean=df_clean.fillna(0)

In [186]:
#Removing of the departure delays rows where I don't have the delay data
#df.isnull().sum()
filter1= ~df_clean['DEP_DELAY'].isnull()
df_clean=df_clean[filter1]

In [187]:
#Removing of the arrival delays where I don't have the delay data
filter2=~df_clean['ARR_DELAY'].isnull()
df_clean=df_clean[filter2]

In [192]:

'''df_clean1=df_clean
num = df_clean1._get_numeric_data()
num[num < 0] = 0
'''
df_clean.shape

(7213446, 22)

# Adding of the month column

In [193]:
import re
def get_month(x):
    return re.findall(r'-[0-9]{2}',x)[0].split('-')[1]
df_clean['MONTH']=df_clean['FL_DATE'].apply(get_month)

In [194]:
df_clean.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_TIME,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,MONTH
0,2018-01-01,UA,2429,EWR,DEN,1517,1512.0,-5.0,1745,1722.0,...,268.0,250.0,225.0,1605.0,0.0,0.0,0.0,0.0,0.0,01
1,2018-01-01,UA,2427,LAS,SFO,1115,1107.0,-8.0,1254,1230.0,...,99.0,83.0,65.0,414.0,0.0,0.0,0.0,0.0,0.0,01
2,2018-01-01,UA,2426,SNA,DEN,1335,1330.0,-5.0,1649,1636.0,...,134.0,126.0,106.0,846.0,0.0,0.0,0.0,0.0,0.0,01
3,2018-01-01,UA,2425,RSW,ORD,1546,1552.0,6.0,1756,1754.0,...,190.0,182.0,157.0,1120.0,0.0,0.0,0.0,0.0,0.0,01
4,2018-01-01,UA,2424,ORD,ALB,630,650.0,20.0,922,936.0,...,112.0,106.0,83.0,723.0,0.0,0.0,0.0,0.0,0.0,01


# Airport info API call to find infos about the country given the iata code

In [130]:
#all posSible iata labels
iataLabels=df_clean['ORIGIN'].value_counts().index

The API provides the data for eash airport. I create 2 new columns to store the names of arrival and departure countries

In [139]:
import os
from dotenv import load_dotenv
load_dotenv()

# Load the apikey
apiKey = os.getenv("APIKEY")
print("WE HAVE APIKEY") if apiKey else print("NO APIKEY FOUND")

WE HAVE APIKEY


In [136]:

import requests

url = "https://airport-info.p.rapidapi.com/airport"

iata_dict=dict()

for ia in iataLabels:
    querystring = {"iata": ia }

    headers = {
        'x-rapidapi-host': "airport-info.p.rapidapi.com",
        'x-rapidapi-key': APIKEY
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    iata_dict[ ia ] = response.text
    

In [144]:
from ast import literal_eval

def find_country(iata):
    if iata in iata_dict.keys():
        python_dict = literal_eval(iata_dict[iata])
        return python_dict['state']
    else:
        return 'unknown'
'''
df_clean1['DEP_STATE']=df_clean1['ORIGIN'].apply(find_country)
df_clean1['ARR_STATE']=df_clean1['DEST'].apply(find_country)'''

In [183]:
df_iata.head(10)

,DEST,AIRPORT_NAME,COUNTRY,ARR_COUNTRY,LOCATION
0,ATL,Hartsfield–Jackson Atlanta International Airport,United States,Georgia,"Atlanta, Georgia, United States"
1,ORD,O'Hare International Airport,United States,Illinois,"Chicago, Illinois, United States"
2,DFW,Dallas/Fort Worth International Airport,United States,Texas,"Dallas / Fort Worth, Texas, United States"
3,DEN,Denver International Airport,United States,Colorado,"Denver, Colorado, United States"
4,CLT,Charlotte Douglas International Airport,United States,North Carolina,"Charlotte, North Carolina, United States"
5,LAX,Los Angeles International Airport,United States,California,"Los Angeles, California, United States"
6,SFO,San Francisco International Airport,United States,California,"San Francisco, California, United States"
7,PHX,Phoenix Sky Harbor International Airport,United States,Arizona,"Phoenix, Arizona, United States"
8,IAH,George Bush Intercontinental Airport,United States,Texas,"Houston, Texas, United States"
9,LGA,LaGuardia Airport,United States,New York,"New York City, New York, United States"


In [286]:
#creation of a database from the API relevant information
from ast import literal_eval

lista=[]
for ia in iata_dict.keys():
    python_dict = literal_eval(iata_dict[ia])
    row=(python_dict['iata'],python_dict['name'],python_dict['country'],python_dict['state'],python_dict['location'])
    lista.append(row)
    
df_iata=pd.DataFrame(lista,columns=['DEST','AIRPORT_NAME','COUNTRY','ARR_STATE','LOCATION'])

NameError: name 'df_iata1' is not defined

In [291]:
df_iata1=df_iata[['DEST','AIRPORT_NAME']]
dic= pd.Series(df_iata1.AIRPORT_NAME.values , index=df_iata1.DEST).to_dict()
#Saving the dictionary as json file
import json

json = json.dumps(dic)
f = open("d.json","w")
f.write(json)
f.close()

In [276]:
#join operation between df_clean1 and df_iata

df_clean2=pd.merge(df_clean,df_iata,on='DEST')

In [ ]:
#Including the airline name, found in a csv table

In [249]:
data = pd.read_csv (r"SRC/L_UNIQUE_CARRIERS.csv", encoding='latin-1')
df2 = pd.DataFrame(data)
print(df2.shape)

(1672, 2)


In [255]:
df2.columns

Index(['Code', 'Description'], dtype='object')

In [263]:
#Creation of a dictionary from df2
d=pd.Series(df2.Description.values , index=df2.Code).to_dict()

In [277]:
def name_from_iata(x):
    return d[x]
df_clean2['AIRLINE_NAME']=df_clean2['OP_CARRIER'].apply(name_from_iata)

In [278]:
df_clean2.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_TIME,...,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,MONTH,AIRPORT_NAME,COUNTRY,ARR_COUNTRY,LOCATION,AIRLINE_NAME
0,2018-01-01,UA,2429,EWR,DEN,1517,1512.0,-5.0,1745,1722.0,...,0.0,0.0,0.0,0.0,01,Denver International Airport,United States,Colorado,"Denver, Colorado, United States",United Air Lines Inc.
1,2018-01-01,UA,2426,SNA,DEN,1335,1330.0,-5.0,1649,1636.0,...,0.0,0.0,0.0,0.0,01,Denver International Airport,United States,Colorado,"Denver, Colorado, United States",United Air Lines Inc.
2,2018-01-01,UA,2418,RIC,DEN,1601,1559.0,-2.0,1813,1756.0,...,0.0,0.0,0.0,0.0,01,Denver International Airport,United States,Colorado,"Denver, Colorado, United States",United Air Lines Inc.
3,2018-01-01,UA,2173,IAD,DEN,2217,2235.0,18.0,14,8.0,...,0.0,0.0,0.0,0.0,01,Denver International Airport,United States,Colorado,"Denver, Colorado, United States",United Air Lines Inc.
4,2018-01-01,UA,2166,ANC,DEN,2340,2336.0,-4.0,658,629.0,...,0.0,0.0,0.0,0.0,01,Denver International Airport,United States,Colorado,"Denver, Colorado, United States",United Air Lines Inc.


# Saving of the cleaned Database 

In [279]:
df_clean2.to_csv("SRC/delays_clean.csv")

In [280]:
df_clean3=df_clean2.sample(10000)
df_clean3.to_csv("SRC/delays_clean_small.csv")
